In [22]:
import warnings

warnings.filterwarnings("ignore")

In [23]:
from langchain import hub

prompt_agent = hub.pull('hwchase17/openai-tools-agent')

prompt_agent

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [24]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('/Users/jasper/Downloads/My_first_agents-main/notebooks/.env')

# Verify that the API key is loaded
api_key = os.getenv('OPENAI_API_KEY')

In [25]:
#import os
#from dotenv import load_dotenv

# Load environment variables from .env file
#load_dotenv('/Users/jasper/Downloads/My_first_agents-main/notebooks/.env')

# Verify that the API key is loaded
tavily_api_key = os.getenv('TAVILY_API_KEY')

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0, openai_api_key=api_key)

llm.invoke("HI")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-166bbf16-4547-4c16-ad25-eb3ef3ea6b26-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Agent = LLM + Tools

In [27]:
import subprocess
from langchain_core.tools import tool


@tool
def list_files():
    """List files in current directory"""
    return subprocess.run(["ls"], stdout=subprocess.PIPE).stdout.decode("utf-8")


list_files.invoke({"input": ""})

'1.0-langchain-basics.ipynb\n1.1-langchain-simple-chains.ipynb\n1.2-langchain-runnables-and-lcel.ipynb\n1.3-langchain-simple-agent.ipynb\n2.0-autogen-building-agents.ipynb\n2.1-autogen-building-research-assistant.ipynb\n3.0-langgraph-building-stateful-applications.ipynb\n3.1-langgraph-buid-research-assistant.ipynb\n4.0-rag-agent-langgraph.ipynb\n4.1-autogen-multi-agent-workflows.ipynb\nAutogen Demo Notebook.ipynb\nTool Use.ipynb\nagentic-rag-langgraph.ipynb\n\x1b\x1bassets-resources\x1b[m\x1b[m\n'

In [28]:
llm_with_tool = llm.bind_tools([list_files])

llm_with_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10a9b0590>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10aba0050>, root_client=<openai.OpenAI object at 0x109889210>, root_async_client=<openai.AsyncOpenAI object at 0x10b07b510>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'list_files', 'description': 'List files in current directory', 'parameters': {'properties': {}, 'type': 'object'}}}]}, config={}, config_factories=[])

In [10]:
llm_with_tool.invoke("List files in current directory")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rvyEIE5amrHVqjTkWnEsxad0', 'function': {'arguments': '{}', 'name': 'list_files'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 42, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4ab78625-a55a-4539-a331-b1875e3b1dfd-0', tool_calls=[{'name': 'list_files', 'args': {}, 'id': 'call_rvyEIE5amrHVqjTkWnEsxad0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 10, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [29]:
dictionary_with_tool_names = {
    "list_files": list_files,
}

output = llm_with_tool.invoke("List files in current directory")

if output.tool_calls:
    for tool_call in output.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call['args']
        tool_func = dictionary_with_tool_names[tool_name]
        print(tool_func.invoke(tool_args))

1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
Autogen Demo Notebook.ipynb
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources



Agent = LLM + tools

In [30]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [31]:
tools = [list_files]

agent = create_tool_calling_agent(llm, tools, prompt_agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [32]:
agent_executor.invoke({'input': "List files in current directory"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
Autogen Demo Notebook.ipynb
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources
Here are the files in the current directory:

1. 1.0-langchain-basics.ipynb
2. 1.1-langchain-simple-chains.ipynb
3. 1.2-langchain-runnables-and-lcel.ipynb
4. 1.3-langchain-simple-agent.ipynb
5. 2.0-autogen-building-agents.ipynb
6. 2.1-autogen-building-research-assistant.ipynb
7. 3.0-langgraph-building-stateful-applications.ipynb
8. 3.1-langgraph-buid-research-assistant.ipynb
9. 4.0-rag-agent-langgraph.ipynb
10. 4.1-autogen-multi-agent-workflows.ipynb
11.

{'input': 'List files in current directory',
 'output': 'Here are the files in the current directory:\n\n1. 1.0-langchain-basics.ipynb\n2. 1.1-langchain-simple-chains.ipynb\n3. 1.2-langchain-runnables-and-lcel.ipynb\n4. 1.3-langchain-simple-agent.ipynb\n5. 2.0-autogen-building-agents.ipynb\n6. 2.1-autogen-building-research-assistant.ipynb\n7. 3.0-langgraph-building-stateful-applications.ipynb\n8. 3.1-langgraph-buid-research-assistant.ipynb\n9. 4.0-rag-agent-langgraph.ipynb\n10. 4.1-autogen-multi-agent-workflows.ipynb\n11. Autogen Demo Notebook.ipynb\n12. Tool Use.ipynb\n13. agentic-rag-langgraph.ipynb\n14. assets-resources (directory)'}

In [33]:
agent_executor.invoke({'input': "What kind of files are in the current directory?"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
Autogen Demo Notebook.ipynb
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources
The current directory contains the following files:

1. `1.0-langchain-basics.ipynb`
2. `1.1-langchain-simple-chains.ipynb`
3. `1.2-langchain-runnables-and-lcel.ipynb`
4. `1.3-langchain-simple-agent.ipynb`
5. `2.0-autogen-building-agents.ipynb`
6. `2.1-autogen-building-research-assistant.ipynb`
7. `3.0-langgraph-building-stateful-applications.ipynb`
8. `3.1-langgraph-buid-research-assistant.ipynb`
9. `4.0-rag-agent-langgraph.ipynb`
10. `4.1-autogen-multi

{'input': 'What kind of files are in the current directory?',
 'output': 'The current directory contains the following files:\n\n1. `1.0-langchain-basics.ipynb`\n2. `1.1-langchain-simple-chains.ipynb`\n3. `1.2-langchain-runnables-and-lcel.ipynb`\n4. `1.3-langchain-simple-agent.ipynb`\n5. `2.0-autogen-building-agents.ipynb`\n6. `2.1-autogen-building-research-assistant.ipynb`\n7. `3.0-langgraph-building-stateful-applications.ipynb`\n8. `3.1-langgraph-buid-research-assistant.ipynb`\n9. `4.0-rag-agent-langgraph.ipynb`\n10. `4.1-autogen-multi-agent-workflows.ipynb`\n11. `Autogen Demo Notebook.ipynb`\n12. `Tool Use.ipynb`\n13. `agentic-rag-langgraph.ipynb`\n14. `assets-resources` (directory)'}

In [51]:
import os

#os.environ["TAVILY_API_KEY"] = 'YOUR TAVILY API KEY'
os.environ["TAVILY_API_KEY"] = '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'

In [52]:
from langchain_community.tools import TavilySearchResults

In [56]:
from langchain_community.tools import TavilySearchResults

search_tool = TavilySearchResults(
    max_results = 5,
    search_depth="advanced",
    include_answer=True,
)

search_tool.invoke('What is the capital of France?')

'AttributeError("\'FieldInfo\' object has no attribute \'raw_results\'")'

In [55]:
from langchain_core.tools import tool

# Define the tavily_search_results tool
@tool
def tavily_search_results(query: str, max_results: int = 5, search_depth: str = "advanced", include_answer: bool = True) -> str:
    """
    Perform a search operation based on the query.

    Args:
        query (str): The search query.
        max_results (int): Maximum number of results to return.
        search_depth (str): Depth of the search (e.g., "basic", "advanced").
        include_answer (bool): Whether to include the answer in the results.

    Returns:
        str: The search result.
    """
    # Simulate a search operation
    if "capital of france" in query.lower():
        return "The capital of France is Paris."
    else:
        return "No results found."

# Instantiate the search tool
search_tool = tavily_search_results

# Invoke the search tool with a query
result = search_tool.invoke({
    "query": "What is the capital of France?",
    "max_results": 5,
    "search_depth": "advanced",
    "include_answer": True
})
print(result)

The capital of France is Paris.


In [46]:
import subprocess
from langchain_core.tools import tool

@tool
def list_files():
    """List files in current directory"""
    return subprocess.run(["ls"], stdout=subprocess.PIPE).stdout.decode("utf-8")

@tool
def tavily_search_results(query: str, max_results: int = 5, search_depth: str = "advanced", include_answer: bool = True) -> str:
    """
    Perform a search operation based on the query.

    Args:
        query (str): The search query.
        max_results (int): Maximum number of results to return.
        search_depth (str): Depth of the search (e.g., "basic", "advanced").
        include_answer (bool): Whether to include the answer in the results.

    Returns:
        str: The search result.
    """
    # Simulate a search operation
    if query.lower() == "what is the capital of france?":
        return "The capital of France is Paris."
    else:
        return "No results found."

# Invoke the list_files tool
print(list_files.invoke({"input": ""}))

# Invoke the tavily_search_results tool
search_tool_result = tavily_search_results.invoke({
    "query": "What is the capital of France?",
    "max_results": 5,
    "search_depth": "advanced",
    "include_answer": True
})
print(search_tool_result)

# Assuming you have an instance of ChatOpenAI
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))

# Bind the tools to the language model
llm_with_tools = llm.bind_tools([list_files, tavily_search_results])

# Use the language model with the bound tools
response = llm_with_tools.invoke("List the files in the current directory.")
print(response)

response = llm_with_tools.invoke("What is the capital of France?")
print(response)

1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
Autogen Demo Notebook.ipynb
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources

The capital of France is Paris.
content='' additional_kwargs={'tool_calls': [{'id': 'call_5gTq1WKuGIOecPsSMNiquAGC', 'function': {'arguments': '{}', 'name': 'list_files'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 167, 'total_tokens': 178, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name':

In [37]:
@tool
def create_file(file_path: str, content: str):
    """Create a file with the given content"""
    with open(file_path, 'w') as f:
        f.write(content)
    return f"File created at {file_path} with content: {content}"

tools = [create_file, search_tool]

In [38]:
simple_research_agent = create_tool_calling_agent(llm, tools, prompt_agent)

simple_research_agent_executor = AgentExecutor(agent=simple_research_agent, tools=tools, verbose=True)

simple_research_agent_executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [39]:
from IPython.display import Markdown
output_report = simple_research_agent_executor.invoke({'input': 'Write a summarized report about the main highlights of the Paris Olympics 2024 and save it locally as report-olympics-2024.md'})

Markdown(output_report['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 highlights'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 highlights summary'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 news highlights'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris 2024 Olympics key highlights'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris 2024 Olympics overview'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 updates'}`


AttributeError("'FieldInfo' object has no attribute 'ra

Agent stopped due to max iterations.

In [40]:
with open('report-olympics-2024.md', 'r') as f:
    content_report = f.read()
    
Markdown(content_report)

FileNotFoundError: [Errno 2] No such file or directory: 'report-olympics-2024.md'

In [49]:
import subprocess
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from IPython.display import Markdown

# Define the list_files tool
@tool
def list_files():
    """List files in current directory"""
    return subprocess.run(["ls"], stdout=subprocess.PIPE).stdout.decode("utf-8")

# Define the tavily_search_results tool
@tool
def tavily_search_results(query: str, max_results: int = 5, search_depth: str = "advanced", include_answer: bool = True) -> str:
    """
    Perform a search operation based on the query.

    Args:
        query (str): The search query.
        max_results (int): Maximum number of results to return.
        search_depth (str): Depth of the search (e.g., "basic", "advanced").
        include_answer (bool): Whether to include the answer in the results.

    Returns:
        str: The search result.
    """
    # Simulate a search operation
    if query.lower() == "what is the capital of France?":
        return "The capital of France is Paris."
    else:
        return "No results found."

# Define the create_file tool
@tool
def create_file(file_path: str, content: str) -> str:
    """Create a file with the given content"""
    with open(file_path, 'w') as f:
        f.write(content)
    return f"File created at {file_path} with content: {content}"

# Initialize the language model
llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))

# Bind the tools to the language model
tools = [list_files, tavily_search_results, create_file]
llm_with_tools = llm.bind_tools(tools)

# Define the prompt template for the agent
prompt_template = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],
    template="""
    You are a helpful assistant. Use the tools provided to answer the user's questions and perform tasks.
    {agent_scratchpad}
    User: {input}
    """
)

# Create the tool-calling agent
simple_research_agent = create_tool_calling_agent(llm_with_tools, tools, prompt_template)

# Create the agent executor
simple_research_agent_executor = AgentExecutor(agent=simple_research_agent, tools=tools, verbose=True)

# Invoke the agent executor with a task
output_report = simple_research_agent_executor.invoke({
    'input': 'Write a summarized report about the main highlights of the Paris Olympics 2024 and save it locally as report-olympics-2024.md'
})

# Display the output report
Markdown(output_report['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Paris Olympics 2024 highlights', 'max_results': 5, 'search_depth': 'advanced', 'include_answer': True}`


No results found.
Invoking: `create_file` with `{'file_path': 'report-olympics-2024.md', 'content': '# Summary Report: Highlights of the Paris Olympics 2024\n\nThe Paris Olympics 2024, scheduled from July 26 to August 11, 2024, promises to be a spectacular event showcasing athletic excellence and cultural celebration. Here are the main highlights expected from the event:\n\n## 1. Historic Venues\n- The Games will take place in iconic locations across Paris, including the Stade de France, the historic heart of the city, and the stunning backdrop of the Seine River for events like swimming and triathlon.\n\n## 2. New Sports and Events\n- The 2024 Olympics will introduce new sports to the Olympic program, including breakdancing, which will make its debut, alongside returning sports like skatebo

I have created a summarized report about the main highlights of the Paris Olympics 2024. The report has been saved as `report-olympics-2024.md`. Here are the key points included in the report:

# Summary Report: Highlights of the Paris Olympics 2024

The Paris Olympics 2024, scheduled from July 26 to August 11, 2024, promises to be a spectacular event showcasing athletic excellence and cultural celebration. Here are the main highlights expected from the event:

## 1. Historic Venues
- The Games will take place in iconic locations across Paris, including the Stade de France, the historic heart of the city, and the stunning backdrop of the Seine River for events like swimming and triathlon.

## 2. New Sports and Events
- The 2024 Olympics will introduce new sports to the Olympic program, including breakdancing, which will make its debut, alongside returning sports like skateboarding and sport climbing.

## 3. Sustainability Initiatives
- Paris 2024 aims to set new standards in sustainability, focusing on eco-friendly practices in construction, waste management, and transportation to minimize the environmental impact of the Games.

## 4. Cultural Celebrations
- The Olympics will feature vibrant cultural events, including the opening ceremony, which will take place along the Seine, allowing for a unique experience for athletes and spectators alike.

## 5. Global Participation
- Athletes from around the world will compete in various sports, fostering a spirit of unity and sportsmanship, as well as showcasing diverse cultures on a global stage.

## Conclusion
The Paris Olympics 2024 is set to be a remarkable event, blending sports, culture, and sustainability, making it a milestone in Olympic history. 

If you need anything else, feel free to ask!